## Resilient Distributed Datasets (RDDs) - Lab

Resilient Distributed Datasets (RDD) are fundamental data structures of Spark. An RDD is essentially the Spark representation of a set of data, spread across multiple machines, with APIs to let you act on it. An RDD can come from any data source, e.g. text files, a database, a JSON file, etc.


## Objectives

You will be able to:

- Apply the map(func) transformation to a given function on all elements of an RDD in different partitions 
- Apply a map transformation for all elements of an RDD 
- Compare the difference between a transformation and an action within RDDs 
- Use collect(), count(), and take() actions to trigger spark transformations  
- Use filter to select data that meets certain specifications within an RDD 
- Set number of partitions for parallelizing RDDs 
- Create RDDs from Python collections 


## What are RDDs? 

To get a better understanding of RDDs, let's break down each one of the components of the acronym RDD:

Resilient: RDDs are considered "resilient" because they have built-in fault tolerance. This means that even if one of the nodes goes offline, RDDs will be able to restore the data. This is already a huge advantage compared to standard storage. If a standard computer dies while performing an operation, all of its memory will be lost in the process. With RDDs, multiple nodes can go offline, and the action will still be held in working memory.

Distributed: The data is contained on multiple nodes of a cluster-computing operation. It is efficiently partitioned to allow for parallelism.

Dataset: The dataset has been * partitioned * across the multiple nodes. 

RDDs are the building block upon which more high-level Spark operations are based upon. Chances are, if you are performing an action using Spark, the operation involves RDDs. 



Key Characteristics of RDDs:

- Immutable: Once an RDD is created, it cannot be modified. 
- Lazily Evaluated: RDDs will not be evaluated until an action is triggered. Essentially, when RDDs are created, they are programmed to perform some action, but that function will not get activated until it is explicitly called. The reason for lazy evaluation is that allows users to organize the actions of their Spark program into smaller actions. It also saves unnecessary computation and memory load.
- In-Memory: The operations in Spark are performed in-memory rather than in the database. This is what allows Spark to perform fast operations with very large quantities of data.




### RDD Transformations vs Actions

In Spark, we first create a __base RDD__ and then apply one or more transformations to that base RDD following our processing needs. Being immutable means, **once an RDD is created, it cannot be changed**. As a result, **each transformation of an RDD creates a new RDD**. Finally, we can apply one or more **actions** to the RDDs. Spark uses lazy evaluation, so transformations are not actually executed until an action occurs.


<img src="./images/rdd_diagram.png" width=500>

### Transformations

Transformations create a new dataset from an existing one by passing each dataset element through a function and returning a new RDD representing the results. In short, creating an RDD from an existing RDD is ‘transformation’.
All transformations in Spark are lazy. They do not compute their results right away. Instead, they just remember the transformations applied to some base dataset (e.g. a file). The transformations are only computed when an action requires a result that needs to be returned to the driver program.
A transformation is an RDD that returns another RDD, like map, flatMap, filter, reduceByKey, join, cogroup, etc.

### Actions
Actions return final results of RDD computations. Actions trigger execution using lineage graph to load the data into original RDD and carry out all intermediate transformations and return the final results to the driver program or writes it out to the file system. An action returns a value (to a Spark driver - the user program).

Here are some key transformations and actions that we will explore.


| Transformations   | Actions       |
|-------------------|---------------|
| map(func)         | reduce(func)  |
| filter(func)      | collect()     |
| groupByKey()      | count()       |
| reduceByKey(func) | first()       |
| mapValues(func)   | take()        |
| sample()          | countByKey()  |
| distinct()        | foreach(func) |
| sortByKey()       |               |


Let's see how transformations and actions work through a simple example. In this example, we will perform several actions and transformations on RDDs in order to obtain a better understanding of Spark processing. 

### Create a Python collection 

We need some data to start experimenting with RDDs. Let's create some sample data and see how RDDs handle it. To practice working with RDDs, we're going to use a simple Python list.

- Create a Python list `data` of integers between 1 and 1000 using the `range()` function. 
- Sanity check: confirm the length of the list (it should be 1000)

In [1]:
data = list(range(1,1001))
len(data)

# 1000

1000

### Initialize an RDD

When using Spark to make computations, datasets are treated as lists of entries. Those lists are split into different partitions across different cores or different computers. Each list of data held in memory is a partition of the RDD. The reason why Spark is able to make computations far faster than other big data processing languages is that it allows all data to be stored __in-memory__, which allows for easy access to the data and, in turn, high-speed processing. Here is an example of how the alphabet might be split into different RDDs and held across a distributed collection of nodes:

<img src ="./images/partitions_1.png" width ="500">  
To initialize an RDD, first import `pyspark` and then create a SparkContext assigned to the variable `sc`. Use `'local[*]'` as the master.

In [2]:
import pyspark
sc = pyspark.SparkContext("local[*]")

Once you've created the SparkContext, you can use the `.parallelize()` method to create an RDD that will distribute the list of numbers across multiple cores. Here, create one called `rdd` with 10 partitions using `data` as the collection you are parallelizing.

In [3]:
rdd = sc.parallelize(data, 10)
print(type(rdd))
# <class 'pyspark.rdd.RDD'>

<class 'pyspark.rdd.RDD'>


Determine how many partitions are being used with this RDD with the `.getNumPartitions()` method.

In [4]:
rdd.getNumPartitions()
# 10

10

### Basic descriptive RDD actions

Let's perform some basic operations on our RDD. In the cell below, use the methods:
* `count`: returns the total count of items in the RDD 
* `first`: returns the first item in the RDD
* `take`: returns the first `n` items in the RDD
* `top`: returns the top `n` items
* `collect`: returns everything from your RDD


It's important to note that in a big data context, calling the collect method will often take a very long time to execute and should be handled with care!

In [5]:
# count
rdd.count()

1000

In [6]:
# first
rdd.first()

1

In [7]:
# take
rdd.take(5)

[1, 2, 3, 4, 5]

In [8]:
# top
rdd.top(5)

[1000, 999, 998, 997, 996]

In [9]:
# collect
rdd.collect()

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

## Map functions

Now that you've been working a little bit with RDDs, let's make this a little more interesting. Imagine you're running a hot new e-commerce startup called BuyStuff, and you're trying to track of how much it charges customers from each item sold. In the next cell, we're going to create simulated data by multiplying the values 1-1000 with a random number from 0-1.

In [10]:
import random
import numpy as np

nums = np.array(range(1, 1001))
sales_figures = nums * np.random.rand(1000)
sales_figures

array([3.28903265e-01, 1.91675618e+00, 2.95318020e+00, 3.27804625e-01,
       3.28584825e+00, 4.26413356e+00, 7.51633261e-01, 4.85781607e+00,
       6.96650079e-01, 4.74579112e+00, 5.75289595e+00, 1.13923011e+01,
       8.14711059e+00, 1.06309976e+01, 1.28100415e+01, 8.02392540e+00,
       1.17520364e+01, 2.43494275e+00, 6.82630470e+00, 7.46205716e+00,
       2.03049858e+01, 1.29097486e+01, 1.88746307e+01, 7.99628063e+00,
       1.98825749e+01, 1.58202899e+01, 1.29437375e+01, 2.37343091e+00,
       2.47561446e+01, 2.29444583e+01, 9.22123604e+00, 2.13833744e+01,
       4.94742100e+00, 3.13623040e+01, 1.20870937e+01, 2.95498598e+01,
       3.20954635e+01, 2.39660922e+01, 9.36689073e-01, 9.54728092e+00,
       1.32533688e+01, 2.08081418e+01, 3.21829016e+01, 1.76375324e+01,
       4.29240342e+01, 4.22026475e+01, 2.66876701e+01, 3.09721557e+01,
       1.12146436e+01, 2.31100716e+01, 2.23243065e+01, 6.16070311e+00,
       3.32216357e+00, 1.39544614e+01, 3.21144903e+01, 3.06557695e+01,
      

We now have sales prices for 1000 items currently for sale at BuyStuff. Now create an RDD called `price_items` using the newly created data with 10 slices. After you create it, use one of the basic actions to see what's in the RDD.

In [11]:
price_items = sc.parallelize(sales_figures, 10)
price_items.top(10)

[982.4279903347339,
 964.1715227726828,
 932.0120274429266,
 906.6935263769471,
 893.6771901454432,
 892.587719909648,
 891.7951534208207,
 889.0609945225204,
 868.0943612276227,
 862.7264141735801]

Now let's perform some operations on this simple dataset. To begin with, create a function that will take into account how much money BuyStuff will receive after sales tax has been applied (assume a sales tax of 8%). To make this happen, create a function called `sales_tax()` that returns the amount of money our company will receive after the sales tax has been applied. The function will have this parameter:

* `item`: (float) number to be multiplied by the sales tax.


Apply that function to the rdd by using the `.map()` method and assign it to a variable `renenue_minus_tax`

In [12]:
def sales_tax(num):
    return num/1.08

revenue_minus_tax = price_items.map(sales_tax)

Remember, Spark has __lazy evaluation__, which means that the `sales_tax()` function is a transformer that is not executed until you call an action. Use one of the collection methods to execute the transformer now a part of the RDD and observe the contents of the `revenue_minus_tax` rdd.

In [13]:
# perform action to retrieve rdd values
revenue_minus_tax.top(10)

[909.655546606235,
 892.7514099747062,
 862.9740994841912,
 839.5310429416177,
 827.4788797642992,
 826.4701110274517,
 825.7362531674265,
 823.2046245578892,
 803.7910752107617,
 798.820753864426]

### Lambda Functions

Note that you can also use lambda functions if you want to quickly perform simple operations on data without creating a function. Let's assume that BuyStuff has also decided to offer a 10% discount on all of their items on the pre-tax amounts of each item. Use a lambda function within a `.map()` method to apply the additional 10% loss in revenue for BuyStuff and assign the transformed RDD to a new RDD called `discounted`.

In [14]:
discounted = revenue_minus_tax.map(lambda x: x*0.9)

In [15]:
discounted.take(10)

[0.2740860541210104,
 1.5972968132334546,
 2.460983502389657,
 0.27317052116923624,
 2.7382068769256325,
 3.553444637170658,
 0.6263610504350199,
 4.0481800552755915,
 0.5805417322497602,
 3.9548259335131943]

## Chaining Methods

You are also able to chain methods together with Spark. In one line, remove the tax and discount from the revenue of BuyStuff and use a collection method to see the 15 costliest items.

In [16]:
price_items.map(sales_tax).map(lambda x: x*0.9).top(15)

[818.6899919456115,
 803.4762689772356,
 776.6766895357721,
 755.5779386474559,
 744.7309917878694,
 743.8230999247065,
 743.1626278506839,
 740.8841621021004,
 723.4119676896855,
 718.9386784779834,
 715.4134107290926,
 714.5213999657491,
 706.0847882709458,
 701.6970831543857,
 697.0229084594562]

## RDD Lineage


We are able to see the full lineage of all the operations that have been performed on an RDD by using the `RDD.toDebugString()` method. As your transformations become more complex, you are encouraged to call this method to get a better understanding of the dependencies between RDDs. Try calling it on the `discounted` RDD to see what RDDs it is dependent on.

In [17]:
discounted.toDebugString()

b'(10) PythonRDD[10] at RDD at PythonRDD.scala:53 []\n |   ParallelCollectionRDD[5] at parallelize at PythonRDD.scala:195 []'

### Map vs. Flatmap

Depending on how you want your data to be outputted, you might want to use `.flatMap()` rather than a simple `.map()`. Let's take a look at how it performs operations versus the standard map. Let's say we wanted to maintain the original amount BuyStuff receives for each item as well as the new amount after the tax and discount are applied. Create a map function that will return a tuple with (original price, post-discount price).

In [18]:
mapped = price_items.map(lambda x: (x, x/1.08 * 0.9))
print(mapped.count())
print(mapped.take(10))

1000
[(0.3289032649452125, 0.2740860541210104), (1.9167561758801457, 1.5972968132334546), (2.9531802028675886, 2.460983502389657), (0.3278046254030835, 0.27317052116923624), (3.2858482523107595, 2.7382068769256325), (4.26413356460479, 3.553444637170658), (0.7516332605220238, 0.6263610504350199), (4.8578160663307095, 4.0481800552755915), (0.6966500786997123, 0.5805417322497602), (4.745791120215833, 3.9548259335131943)]


Note that we have 1000 tuples created to our specification. Let's take a look at how `.flatMap()` differs in its implementation. Use the `.flatMap()` method with the same function you created above.

In [19]:
flat_mapped = price_items.flatMap(lambda x: (x, x/1.08 * 0.9))
print(flat_mapped.count())
print(flat_mapped.take(10))

2000
[0.3289032649452125, 0.2740860541210104, 1.9167561758801457, 1.5972968132334546, 2.9531802028675886, 2.460983502389657, 0.3278046254030835, 0.27317052116923624, 3.2858482523107595, 2.7382068769256325]


Rather than being represented by tuples, all of the  values are now on the same level. When we are trying to combine different items together, it is sometimes necessary to use `.flatMap()` rather than `.map()` in order to properly reduce to our specifications. This is not one of those instances, but in the upcoming lab, you just might have to use it.

## Filter
After meeting with some external consultants, BuyStuff has determined that its business will be more profitable if it focuses on higher ticket items. Now, use the `.filter()` method to select items that bring in more than $300 after tax and discount have been removed. A filter method is a specialized form of a map function that only returns the items that match a certain criterion. In the cell below:
* use a lambda function within a `.filter()` method to meet the consultant's suggestion's specifications. set `RDD = selected_items`
* calculate the total number of items remaining in BuyStuff's inventory

In [20]:
# use the filter function
selected_items = discounted.filter(lambda x: x>300)

# calculate total remaining in inventory 
selected_items.count()

265

## Reduce

Reduce functions are where you are in some way combing all of the variables that you have mapped out. Here is an example of how a reduce function works when the task is to sum all values:

<img src = "./images/reduce_function.png" width = "600">  


As you can see, the operation is performed within each partition first, after which, the results of the computations in each partition are combined to come up with one final answer.  

Now it's time to figure out how much money BuyStuff would make from selling one of all of its items after they've reduced their inventory. Use the `.reduce()` method with a lambda function to add up all of the values in the RDD. Your lambda function should have two variables. 

In [21]:
selected_items.reduce(lambda x,y: x+y)

123757.22998395488

The time has come for BuyStuff to open up shop and start selling its goods. It only has one of each item, but it's allowing 50 lucky users to buy as many items as they want while they remain in stock. Within seconds, BuyStuff is sold out. Below, you'll find the sales data in an RDD with tuples of (user, item bought).

In [22]:
import random
random.seed(42)
# generating simulated users that have bought each item
sales_data = selected_items.map(lambda x: (random.randint(1, 50), x))

sales_data.take(7)

[(26, 309.434711212615),
 (31, 305.313066057473),
 (30, 351.02997634859867),
 (45, 347.58441582893914),
 (10, 328.58433591957026),
 (38, 334.9802853900219),
 (3, 342.5692045948199)]

It's time to determine some basic statistics about BuyStuff users.

Let's start off by creating an RDD that determines how much each user spent in total.
To do this we can use a method called `.reduceByKey()` to perform reducing operations while grouping by keys. After you have calculated the total, use the `.sortBy()` method on the RDD to rank the users from the highest spending to the least spending. 

In [23]:
# calculate how much each user spent
total = sales_data.reduceByKey(lambda x,y: x+y)

In [24]:
# sort the users from highest to lowest spenders
total.sortBy(lambda x: x[1], False).collect()

[(6, 6026.970208084914),
 (16, 4954.374340925326),
 (15, 4830.816321832742),
 (21, 4674.287532753501),
 (13, 4623.292187407753),
 (25, 4310.842398463103),
 (4, 4265.760932856415),
 (20, 4083.1903858963237),
 (10, 3787.3475460216187),
 (43, 3666.4210963114324),
 (19, 3326.4664391737),
 (11, 3115.272394901687),
 (32, 3057.3945495838125),
 (38, 3000.747145866509),
 (2, 2921.5019113743247),
 (41, 2865.3868908085547),
 (17, 2824.64437600506),
 (31, 2821.993511619365),
 (34, 2590.4293470784946),
 (44, 2468.7164150048156),
 (48, 2414.921659664707),
 (30, 2304.168273600859),
 (12, 2293.861984513298),
 (3, 2251.405145075347),
 (36, 2246.602874794858),
 (22, 2220.9198914180315),
 (45, 2153.7853059811796),
 (9, 2128.8175548691515),
 (39, 2035.393900163106),
 (27, 2002.1037679545416),
 (5, 1929.0043630872449),
 (24, 1924.5572266428878),
 (49, 1845.4005364893537),
 (33, 1831.1583581130767),
 (37, 1825.9484257388067),
 (40, 1806.8479397511326),
 (28, 1779.547925377392),
 (42, 1687.7616412306847),
 (

Next, let's determine how many items were bought per user. This can be solved in one line using an RDD method. After you've counted the total number of items bought per person, sort the users from most number of items bought to least number of items. Time to start a customer loyalty program!

In [25]:
sorted(sales_data.countByKey().items(), key=lambda x: x[1], reverse=True)

[(48, 11),
 (14, 9),
 (35, 9),
 (24, 9),
 (15, 8),
 (47, 8),
 (1, 8),
 (31, 8),
 (25, 7),
 (33, 7),
 (11, 7),
 (27, 7),
 (5, 7),
 (30, 7),
 (7, 7),
 (37, 6),
 (18, 6),
 (29, 6),
 (39, 6),
 (16, 6),
 (38, 6),
 (6, 5),
 (46, 5),
 (34, 5),
 (21, 5),
 (32, 5),
 (49, 5),
 (40, 5),
 (9, 5),
 (36, 5),
 (4, 5),
 (17, 5),
 (45, 4),
 (19, 4),
 (42, 4),
 (41, 4),
 (26, 4),
 (13, 4),
 (20, 4),
 (23, 3),
 (43, 3),
 (2, 3),
 (10, 3),
 (12, 3),
 (3, 3),
 (8, 2),
 (50, 2),
 (22, 2),
 (44, 2),
 (28, 1)]

### Additional Reading

- [The original paper on RDDs](https://cs.stanford.edu/~matei/papers/2012/nsdi_spark.pdf)
- [RDDs in Apache Spark](https://data-flair.training/blogs/create-rdds-in-apache-spark/)
- [Programming with RDDs](https://runawayhorse001.github.io/LearningApacheSpark/rdd.html)
- [RDD Transformations and Actions Summary](https://www.analyticsvidhya.com/blog/2016/10/using-pyspark-to-perform-transformations-and-actions-on-rdd/)

## Summary

In this lab we went through a brief introduction to RDD creation from a Python collection, setting a number of logical partitions for an RDD and extracting lineage. We also used transformations and actions to perform calculations across RDDs on a distributed setup. In the next lab, you'll get the chance to apply these transformations on different books to calculate word counts and various statistics.
